# Query DB for Emails

In [ ]:
%load_ext autoreload
%autoreload 2

from MySQL_Scripts.modules.acquiring_db_emails import query_database, drop_unsubscribed


#Query the DB for the customer base that you are narrowing down to, and drop unsubscribed emails
mlb = [
    'Angels', 'Astros', 'Athletics', 'Blue Jays', 'Braves', 'Brewers', 'Cardinals', 'Cubs', 'Diamondbacks', 'Dodgers',
    'Giants', 'Indians', 'Mariners', 'Marlins', 'Mets', 'Nationals', 'Orioles', 'Padres', 'Phillies', 'Pirates',
    'Rangers', 'Rays', 'Red Sox', 'Reds', 'Rockies', 'Royals', 'Tigers', 'Twins', 'White Sox', 'Yankees', 'Baseball', 'MLB'
]

# Create the SQL query string
query = f'''
    SELECT * 
    FROM email_prospecting
    WHERE {' OR '.join([f"LOWER(product_name) LIKE '%{x.lower()}%'" for x in mlb])};
'''

# Assuming you have a function query_database defined
df = query_database(query)
df = drop_unsubscribed(df)

#This is then references from a csv in the sends

# Query Local CSV for School Emails

In [10]:
%load_ext autoreload
%autoreload 2
from Sending_Emails.modules import KC_schools
import pandas as pd
pd.set_option('display.max_columns', None)

df = KC_schools.read_in()
df = KC_schools.filter_emails_by_sport(df, ['Baseball', 'Softball'])

#DROP DUPLICATES


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
df = df.loc[1493:].reset_index(drop = True)

# df.loc[df['email'].str.contains('herrmanns')]

# Initialize A Class in Order to Call Process Function and Clean up Main Code

In [13]:
# this allows all variables to be in one area. Allows for better swap in between templates. 
# 
%load_ext autoreload
%autoreload 2

from config import imap_password_customplanet, db_username, db_password
import pandas as pd
import numpy as np
from datetime import datetime
import time
import logging
import os
from Sending_Emails.modules.sends import EmailConfig
from Sending_Emails.modules.sends import SendMail
from Sending_Emails.modules.html_email_strings.schools_sport_focus import get_template  #THis import dictates template


#create log dir and file
logpath_creation = os.getcwd() + '\\Logs'
if not os.path.exists(logpath_creation):
    os.makedirs(logpath_creation)

logging.basicConfig(filename= logpath_creation + '\\Email_Sender.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)


# --------------------------------------Initialize All Variables--------------------------------

email_config = EmailConfig(
                            EMAIL_ADDRESS_FROM = 'team@customplanet.com',
                            EMAIL_PASS = imap_password_customplanet,
    
    
                            #This connection sustains throughout the email blast loop
                            SMTP_CONN = SendMail.get_smtp_connection('team@customplanet.com', imap_password_customplanet),
                            df = df, 
                            #This df is what is created from DW, or Local CSV

                           contact_column = 'email',  #should always be email. 
                           sport = 'Baseball',
                           email_campaign_name = 'Offical MLB Jerseys in Stock!!', 
                           email_subject_line = 'Local Supplier for Baseball Uniforms & Apparel',
                        #    template = get_template(school=None, sport=None),
            
                           #Not necessary for Send Script. Only Scrape Script
                            server = 'emailcampaign.c9vhoi6ncot7.us-east-1.rds.amazonaws.com',
                            database = 'emailcampaign',
                            table_name = 'email_history'
                           )


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Calling the Function to Send Emails Iteratively

In [14]:
#The logic of this function works by declaring the email_history variable which is returned from the process function
# This is a dataframe of the emails who it has been sent to. Also is written to output.csv


def blast(email_config, test=False):

    df = email_config.df

    # Get the next 50 emails iteratively and send every 60 seconds with a new SMTP connection
    # get_next_50 is what ends this function. It will break once end_point has surpassed.
    while len(df) >= 1:

        next_50 = SendMail.get_next_50(df)

        new_point = next_50.index[0]
        end_point = next_50.index[-1]   
        print(new_point, end_point) 
        logging.info(f'\n\nStarting email send at index {new_point}, and ending at index {end_point}')

         # Check if the end_point surpasses 1500
        if end_point >= 1500:
            logging.info('Reached the end_point limit of 1500. Exited the loop.')
            print('Reached the end_point limit of 1500. Exited the loop.')
            break
        
        # Process the emails
        email_history = SendMail.process(next_50, email_config, test)
        email_history.to_csv('output.csv', index = False)
        
      
        interval_seconds = 30
        # Wait for the specified interval before the next iteration
        try:
            time.sleep(interval_seconds)
            logging.info(f'Sleeping for {interval_seconds} seconds completed')
            print(f'Sleeping for {interval_seconds} seconds')
        except:
            logging.info('Issue with the sleep')
            print('Issue with the sleep')
    
    os.remove('output.csv')


#If test ARG is True it will keep sending to personal email
#& output.csv must not exist at the beginning of new runs
    
blast(email_config, test=False)

#Insert try except for catching 
# IndexError

Output is not created, first run of 50 emails preparing to be sent
0 49
0 Congress Middle (Park Hill)
1 Shawanoe Elementary School
2 Lucy Franklin Elementary
3 BSHS Freshman Campus
4 Rockhurst High School (KCMO)
5 Crestview Elementary School
6 Gladden Elementary (Belton)
7 Smith-Hale Middle (Hickman Mills-KCMO)
8 Liberty Oaks Elementary School (Liberty)
9 Liberty Oaks Elementary
Skipping email to jeanne.hinnenkamp@lps53.org as it has already been processed.
10 The Pembroke Hill School (KCMO)
11 Indian Woods Middle School
12 Pleasant Lea Middle (Lee's Summit)
13 Summit Lakes Middle (Lee's Summit)
Recipient error for whitney.morehead: {'whitney.morehead': (553, b'5.1.3 The recipient address <whitney.morehead> is not a valid RFC 5321\n5.1.3 address. For more information, go to\n5.1.3  https://support.google.com/a/answer/3221692 and review RFC 5321\n5.1.3 specifications. b10-20020a170903228a00b001dcb4ae9563sm10892385plh.33 - gsmtp')}
14 North Kansas City High (NKCSD)
15 Olathe Northwest Hi

IndexError: index 0 is out of bounds for axis 0 with size 0